# Classifying Bank Transactions

### Overview:

My objective with this notebook is to demonstrate my solution to the data science challenge given, to input 
bank transactional data and return a data structure summarizing a user's regularly recurring transactions as output.\
\
My solution is a rule based proceedure, it parses information from a bank transaction description to group similarly named transactions together. Before a series of functions which filter out groups unlikely to be recurring. I used a rule based model here because its easily interpretable, explainable and our assumptions known. As well as being fast. However, we shall see that, with the complexity of the problem it makes patching up special cases difficult. As well as not providing more descriptive information which could be useful, that a more complex model could provide. \
\
I provide my first model solution, before reviewing, then making improvements before the final model. I review my final model, before concluding and discussing my next steps - if they were to be taken.

# Table of contents
1. [Loading, Sorting and Previewing](#loading)
2.    [First Model](#firstmodel)
3.    [First Model Review](#firstmodelnreview)
4.    [Final Model](#finalmodel)
5.    [Final Model Review](#finalmodelreview)
6.    [Conclusion & Next Steps](#nextsteps)

## Loading, Sorting and Previewing.<a name="loading"></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [2]:
path_to_data="example_data.csv"

In [3]:
def preprocessing(path_to_data):
    """Returns sorted transactions, in ascending order by 'made_on' date after loading data."""
    transactions = pd.read_csv(path_to_data, index_col=0)
    transactions['made_on'] = pd.to_datetime(transactions.made_on)
    transactions = transactions.sort_values('made_on')
    return transactions

In [4]:
transactions = preprocessing(path_to_data)

In [5]:
transactions.head()

,user_id,made_on,original_description,amount_cents,income
617,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-22,HAWKS CLUB ON 20 JUL CLP,3000,False
891,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-22,SAINSBURYS S/MKTS ON 21 JUL CLP,505,False
940,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,WASABI CAMBRIDGE ON 22 JUL CLP,1710,False
616,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,NYA*Cambridge Vend ON 22 JUL BCC,250,False
941,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,AROMI CAFFE LTD ON 22 JUL CLP,410,False


In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1210 entries, 617 to 1208
Data columns (total 5 columns):
user_id                 1210 non-null object
made_on                 1210 non-null datetime64[ns]
original_description    1210 non-null object
amount_cents            1210 non-null int64
income                  1210 non-null bool
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 48.4+ KB


## First Model.<a name="firstmodek"></a>

In [7]:
def parse_original_description(transations):
    """Returns transactions after adding new column equal to first word of 'original_description'.
    Splits 'original_description' by " " and "*" characters.
    """
    original_description_split_astrix_space = transactions['original_description'].str.split(" |\*", n=3, expand=True)
    transactions['original_description_first_word'] = original_description_split_astrix_space[0]
    
    return transactions

In [8]:
transactions = parse_original_description(transactions)
transactions.head()

,user_id,made_on,original_description,amount_cents,income,original_description_first_word
617,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-22,HAWKS CLUB ON 20 JUL CLP,3000,False,HAWKS
891,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-22,SAINSBURYS S/MKTS ON 21 JUL CLP,505,False,SAINSBURYS
940,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,WASABI CAMBRIDGE ON 22 JUL CLP,1710,False,WASABI
616,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,NYA*Cambridge Vend ON 22 JUL BCC,250,False,NYA
941,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-07-23,AROMI CAFFE LTD ON 22 JUL CLP,410,False,AROMI


In [9]:
def remove_groups_too_small(transactions, info, lower_minimum=3):
    """Returns transactions after removing groups with less than three transactions.
    
    input info: whether or not to display processing information,
    input lower_minimum: lower limit on the size of a group to not be removed, default = 3.
    """
    # save transactions and its respective groups to track changes.
    input_transactions = transactions
    transactions_grouped = transactions.groupby('original_description_first_word')
    input_transactions_grouped = transactions_grouped
    
    # if group has less than the lower_minimum number transactions remove group from transactions dataframe.
    for name, group in transactions_grouped:
        if group['user_id'].count() < lower_minimum:
            transactions = transactions.drop(transactions_grouped.get_group(name).index)

    transactions_grouped = transactions.groupby('original_description_first_word')
    
    # print information
    if info:
        print("Old number of transactions: {} \n\
        Old number of groups: {} \n\
        Remaining number of transactions: {} \n\
        Remaining number of groups: {} \n"\
        .format(input_transactions['user_id'].count(), len(input_transactions_grouped), transactions['user_id'].count(), len(transactions_grouped)))
    
    return transactions

In [10]:
transactions = remove_groups_too_small(transactions, info=True)

Old number of transactions: 1210 
        Old number of groups: 291 
        Remaining number of transactions: 928 
        Remaining number of groups: 61 



In [11]:
def group_create_time_difference_column(group):
    """Returns a group after adding 'time_difference_forward' which is the time difference in days to 
    next transaction in its group.
    Most recent transaction declared as NaN.
    """
    group = group.reset_index(drop=False)
    group['time_diff_forward'] = np.nan
    
    # loops over transactions in the group, for a transaction calculates time difference (in days) between itself and the next transaction
    for i in group.index:
        t1 = group.loc[i, 'made_on']
        if i+1<len(group.index):
            t2 = group.loc[i+1, 'made_on']
        group['time_diff_forward'].loc[i] = (t2 - t1).days
    
    # sets most recent transaction to NaN
    group.at[group.index[-1], 'time_diff_forward'] = np.nan

    return group

In [12]:
def first_model(transactions, info=True, example_name=None):
    """ First model for get_recurring_transactions() function.
    Returns recurring, a list of dictionaries where each dictionary is of form:
    
    {"entity_name": A common description of the regularly occurring transactions,
    "income": A boolean whether the transaction is debit (True) or credit (False),
    "period_days": The expected interval in days between consecutive transactions for that entity, 
    "typical_amount_cents": The typical amount in cents}
    
    input info: whether or not to display processing information,
    input example_name: name of example to make it easy to select a specific group example for analysis later
    """  
    transactions = parse_original_description(transactions)
    
    # Code to make it easy to investigate specific example e.g 'Spotify'
    if example_name is not None:
        transactions = transactions[transactions['original_description_first_word'] == example_name]
                                                                                           
    transactions = remove_groups_too_small(transactions, info)
    
    recurring = []
    
    for name, group in transactions.groupby('original_description_first_word'):
        group = group_create_time_difference_column(group)
        
        if name == example_name:
            print("{} transactions:\n{}".format(example_name, group))
            
        recurring.append({"entity_name": name, "income": group['income'].all() ,\
                          "period_days": round(group['time_diff_forward'].mean(skipna=True)),\
                          "typical_amount_cents": int(round(group['amount_cents'].mean()))})
    return recurring

In [26]:
transactions = preprocessing(path_to_data)
first_model(transactions)

Old number of transactions: 1210 
        Old number of groups: 291 
        Remaining number of transactions: 928 
        Remaining number of groups: 61 



[{'entity_name': 'AIRBNB',
  'income': False,
  'period_days': 20.0,
  'typical_amount_cents': 66687},
 {'entity_name': 'AMZN',
  'income': False,
  'period_days': 11.0,
  'typical_amount_cents': 1913},
 {'entity_name': 'AMZNMktplace',
  'income': False,
  'period_days': 24.0,
  'typical_amount_cents': 1361},
 {'entity_name': 'APSON',
  'income': True,
  'period_days': 6.0,
  'typical_amount_cents': 73545},
 {'entity_name': 'AQUARIUM',
  'income': False,
  'period_days': 102.0,
  'typical_amount_cents': 1767},
 {'entity_name': 'AROMI',
  'income': False,
  'period_days': 52.0,
  'typical_amount_cents': 675},
 {'entity_name': 'Amazon',
  'income': False,
  'period_days': 40.0,
  'typical_amount_cents': 1081},
 {'entity_name': 'Amazon.co.uk',
  'income': False,
  'period_days': 46.0,
  'typical_amount_cents': 1497},
 {'entity_name': 'BILL',
  'income': False,
  'period_days': 28.0,
  'typical_amount_cents': 25008},
 {'entity_name': 'BON',
  'income': False,
  'period_days': 4.0,
  'typic

In [14]:
first_model(transactions, info=False)

[{'entity_name': 'AIRBNB',
  'income': False,
  'period_days': 20.0,
  'typical_amount_cents': 66687},
 {'entity_name': 'AMZN',
  'income': False,
  'period_days': 11.0,
  'typical_amount_cents': 1913},
 {'entity_name': 'AMZNMktplace',
  'income': False,
  'period_days': 24.0,
  'typical_amount_cents': 1361},
 {'entity_name': 'APSON',
  'income': True,
  'period_days': 6.0,
  'typical_amount_cents': 73545},
 {'entity_name': 'AQUARIUM',
  'income': False,
  'period_days': 102.0,
  'typical_amount_cents': 1767},
 {'entity_name': 'AROMI',
  'income': False,
  'period_days': 52.0,
  'typical_amount_cents': 675},
 {'entity_name': 'Amazon',
  'income': False,
  'period_days': 40.0,
  'typical_amount_cents': 1081},
 {'entity_name': 'Amazon.co.uk',
  'income': False,
  'period_days': 46.0,
  'typical_amount_cents': 1497},
 {'entity_name': 'BILL',
  'income': False,
  'period_days': 28.0,
  'typical_amount_cents': 25008},
 {'entity_name': 'BON',
  'income': False,
  'period_days': 4.0,
  'typic

## First Model Review<a name="firstmodelreview"></a>

In [15]:
first_model(transactions, example_name = 'Spotify')

Old number of transactions: 6 
        Old number of groups: 1 
        Remaining number of transactions: 6 
        Remaining number of groups: 1 

Spotify transactions:
   index                               user_id    made_on  \
0    586  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2018-08-06   
1    889  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2019-06-05   
2    909  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2019-07-05   
3   1016  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2019-08-04   
4   1097  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2019-09-04   
5   1208  9d556712-ff2e-2f9c-73a9-5hf3cce9e389 2019-10-05   

               original_description  amount_cents  income  \
0  Spotify P06EAA1A46 ON 05 AUG BCC           999   False   
1  Spotify P08VAZ2H51 ON 04 JUN CLP           999   False   
2  Spotify P07YAB3H83 ON 04 JUL CLP           999   False   
3          Spotify UK ON 03 AUG BCC           999   False   
4  Spotify P0D97DK26E ON 03 SEP BCC           999   False   
5  Spotify P0D97JL34E ON 04 OCT BCC

[{'entity_name': 'Spotify',
  'income': False,
  'period_days': 85.0,
  'typical_amount_cents': 999}]

## Final Model<a name="finalmodel"></a>

In [16]:
def remove_outliers_time_difference(group, factor, info=True):

    upper_lim = group['time_diff_forward'].mean(skipna=True) + group['time_diff_forward'].std(skipna=True) * factor
    lower_lim = group['time_diff_forward'].mean(skipna=True) - group['time_diff_forward'].std(skipna=True) * factor

    # I don't want to remove the most recent transaction where 'time_diff_forward' = Nan. So I will save it in last_row, and remove it temporarily from dataframe.
    last_row = group.tail(1)
    group.drop(group.tail(1).index,inplace=True)

    # remove outliers.
    new_group = group[(lower_lim < group['time_diff_forward']) & (group['time_diff_forward'] < upper_lim) & (group['time_diff_forward'] != np.nan) ]
    
    if info:
        print("For {} removed {} entries. Now {} entries remaining.".format(group['original_description_first_word'].iloc[0], group['user_id'].count() - new_group['user_id'].count(), new_group['user_id'].count()+1))

    group = new_group

    # Append most recent transcation where 'time_diff_forward' = Nan.
    group = group.append(last_row, ignore_index=True)
    
    return group

In [17]:
def remove_outliers_amount_cents(group, factor, info=True):

    upper_lim = group['amount_cents'].mean() + group['amount_cents'].std() * factor
    lower_lim = group['amount_cents'].mean() - group['amount_cents'].std() * factor

    # remove outliers.
    new_group = group[(lower_lim < group['amount_cents']) & (group['amount_cents'] < upper_lim)]
    
    if info:
        print("For {} removed {} entries. Now {} entries remaining.".format(group['original_description_first_word'].iloc[0], group['user_id'].count() - new_group['user_id'].count(), new_group['user_id'].count()))

    group = new_group
    
    return group

In [18]:
def check_if_recurring_and_label(group):
    if (364 - 4 < group['time_diff_forward'].mean(skipna=True) < 364 + 4):
        label = 'Annually'
        return True, label
    if (364/12 - 3  < group['time_diff_forward'].mean(skipna=True) < 364/12 + 3):
        label = 'Monthly'
        return True, label
    if (14 - 2 < group['time_diff_forward'].mean(skipna=True) < 14 + 2):
        label = 'Bi-weekly'
    if (7 - 1 < group['time_diff_forward'].mean(skipna=True) < 7 + 1):
        label = 'Weekly'
        return True, label
    if (1 + 0.25 < group['time_diff_forward'].mean(skipna=True) < 1 + 0.25):
        label = 'Daily'
        return True, label
    else:
        return False, "Not recurring"

In [19]:
def final_model(transactions, info=True, example_name=None, lower_minimum=3):
    """ Second model for get_recurring_transactions() function.
    Returns recurring, a list of dictionaries where each dictionary is of form:
    
    {"entity_name": A common description of the regularly occurring transactions,
    "income": A boolean whether the transaction is debit (True) or credit (False),
    "period_days": The expected interval in days between consecutive transactions for that entity, 
    "typical_amount_cents": The typical amount in cents}
    
    input info: whether or not to display processing information,
    input example_name: name of example to make it easy to select a specific group example for analysis later,
    input lower_minimum: lower limit on the size of a group to not be removed, default = 3.
    """  
    transactions = parse_original_description(transactions)
    
    # Code to make it easy to investigate specific example e.g 'Spotify'
    if example_name is not None:
        transactions = transactions[transactions['original_description_first_word'] == example_name]
                                                                                           
    transactions = remove_groups_too_small(transactions, info)
    
    recurring = []
    
    for name, group in transactions.groupby('original_description_first_word'):
        group = group_create_time_difference_column(group)
        
        if info:
            print("Removing outliers for {}".format(name))
            
        group = remove_outliers_time_difference(group, factor = 1, info=info)
        group = remove_outliers_amount_cents(group, factor = 1, info = info)
        
        if name == example_name:
            print("{} transactions:\n{}".format(example_name, group))
        
        recurring_boolean, label = check_if_recurring_and_label(group)
        
        if recurring_boolean & (group['user_id'].count() >= lower_minimum):
            recurring.append({"entity_name": label+" "+name.capitalize(), "income": group['income'].all() ,\
                          "period_days": round(group['time_diff_forward'].mean(skipna=True)),\
                          "typical_amount_cents": int(round(group['amount_cents'].mean()))})
    return recurring

In [20]:
transactions = preprocessing(path_to_data)
final_model(transactions)

Old number of transactions: 1210 
        Old number of groups: 291 
        Remaining number of transactions: 928 
        Remaining number of groups: 61 

Removing outliers for AIRBNB
For AIRBNB removed 0 entries. Now 3 entries remaining.
For AIRBNB removed 1 entries. Now 2 entries remaining.
Removing outliers for AMZN
For AMZN removed 0 entries. Now 3 entries remaining.
For AMZN removed 1 entries. Now 2 entries remaining.
Removing outliers for AMZNMktplace
For AMZNMktplace removed 3 entries. Now 5 entries remaining.
For AMZNMktplace removed 1 entries. Now 4 entries remaining.
Removing outliers for APSON
For APSON removed 0 entries. Now 3 entries remaining.
For APSON removed 1 entries. Now 2 entries remaining.
Removing outliers for AQUARIUM
For AQUARIUM removed 0 entries. Now 3 entries remaining.
For AQUARIUM removed 1 entries. Now 2 entries remaining.
Removing outliers for AROMI
For AROMI removed 1 entries. Now 7 entries remaining.
For AROMI removed 2 entries. Now 5 entries remainin

[{'entity_name': 'Monthly Amznmktplace',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 724},
 {'entity_name': 'Monthly Amazon',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 799},
 {'entity_name': 'Monthly Dropbox',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 899},
 {'entity_name': "Monthly Sainsbury's",
  'income': False,
  'period_days': 31.0,
  'typical_amount_cents': 1251},
 {'entity_name': 'Monthly Work',
  'income': True,
  'period_days': 30.0,
  'typical_amount_cents': 178243}]

In [21]:
final_model(transactions, info=False)

[{'entity_name': 'Monthly Amznmktplace',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 724},
 {'entity_name': 'Monthly Amazon',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 799},
 {'entity_name': 'Monthly Dropbox',
  'income': False,
  'period_days': 30.0,
  'typical_amount_cents': 899},
 {'entity_name': "Monthly Sainsbury's",
  'income': False,
  'period_days': 31.0,
  'typical_amount_cents': 1251},
 {'entity_name': 'Monthly Work',
  'income': True,
  'period_days': 30.0,
  'typical_amount_cents': 178243}]

## Final Model Review<a name="finalmodelreview"></a>

In [22]:
transactions = preprocessing(path_to_data)
transactions = parse_original_description(transactions)
transactions[transactions['original_description_first_word'] == 'N'].head()

,user_id,made_on,original_description,amount_cents,income,original_description_first_word
387,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-03-25,N K CONVENIENCE ST ON 23 MAR CLP,338,False,N
368,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-04-01,N K CONVENIENCE ST ON 31 MAR CLP,747,False,N
858,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-05-02,N K CONVENIENCE ST ON 01 MAY CLP,347,False,N
185,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-06-06,N K CONVENIENCE ST ON 05 JUN CLP,249,False,N
123,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-06-24,N K CONVENIENCE ST ON 23 JUN CLP,473,False,N


In [23]:
transactions = preprocessing(path_to_data)
transactions = parse_original_description(transactions)

transactions[transactions['original_description_first_word'] == 'BILL']

,user_id,made_on,original_description,amount_cents,income,original_description_first_word
398,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-03-22,BILL KENNEDY SPLITWISE FT,25255,False,BILL
282,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-05-05,BILL KENNEDY SPLITWISE FT,26165,False,BILL
184,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-06-06,BILL KENNEDY SPLITWISE FT,33714,False,BILL
161,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-06-13,BILL A+SESSA M SWEDEN - ALBERT BBP,14900,True,BILL


In [24]:
transactions = preprocessing(path_to_data)
transactions = parse_original_description(transactions)

transactions[transactions['original_description_first_word'] == 'JOHN']

,user_id,made_on,original_description,amount_cents,income,original_description_first_word
582,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-08-10,JOHN SMITH HELLO BGC,3000,True,JOHN
522,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-11-29,JOHN SMITH MINIGOLF BBP,1400,False,JOHN
517,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-01-21,JOHN SMITH PART DEPOSIT BBP,100000,False,JOHN
497,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-02-04,JOHN SMITH PART DEPOSIT BBP,224858,False,JOHN
496,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-02-04,JOHN SMITH DAVID AIRBNB BBP,6700,False,JOHN
454,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-02-27,JOHN SMITH RENT ETC STO,70236,False,JOHN
384,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-03-27,JOHN SMITH RENT ETC STO,70236,False,JOHN
318,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-04-24,JOHN SMITH RENT ETC STO,70236,False,JOHN
305,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-04-28,JOHN LEWIS ON 27 APR BCC,9430,False,JOHN
299,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-04-29,JOHN LEWIS ON 28 APR BCC,23005,False,JOHN


In [25]:
transactions = preprocessing(path_to_data)
transactions = parse_original_description(transactions)

transactions[transactions['original_description_first_word'] == 'Spotify']

,user_id,made_on,original_description,amount_cents,income,original_description_first_word
586,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2018-08-06,Spotify P06EAA1A46 ON 05 AUG BCC,999,False,Spotify
889,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-06-05,Spotify P08VAZ2H51 ON 04 JUN CLP,999,False,Spotify
909,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-07-05,Spotify P07YAB3H83 ON 04 JUL CLP,999,False,Spotify
1016,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-08-04,Spotify UK ON 03 AUG BCC,999,False,Spotify
1097,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-09-04,Spotify P0D97DK26E ON 03 SEP BCC,999,False,Spotify
1208,9d556712-ff2e-2f9c-73a9-5hf3cce9e389,2019-10-05,Spotify P0D97JL34E ON 04 OCT BCC,999,False,Spotify


## Conclusion & Next Steps <a name="nextsteps"></a>

I used a rule based model here because its easily interpretable, explainable and our assumptions known. The problem with this model is that you have to write rules and assumptions to explain how a human would classify and process the data, which may not be explicitly writable in rules. With this approach the programmer has to think very hard about all the different assumptions and rules they're making. I found lots of problems or special cases to address, on further iterations this could improve a lot, but maybe theres a better way.

The next steps could be to use more complex approaches, which may yield better results. Using a machine learning based model, potentially using natural language processing model to classify whether a subscription, or salary or a friday McDonalds - could work better. To do so would mean creating target features for a training data set which could be done if given more time, and would be a potential next step in builiding a better model.